In [1]:
from medacy.model.model import Model
import pandas as pd
from tqdm import tqdm
import json

## Create bag of words from medical notes

In [2]:
model = Model.load_external('medacy_model_clinical_notes')

In [9]:
# test
aaa = "PREOPERATIVE DIAGNOSIS: Abdominal wall abscess. POSTOPERATIVE DIAGNOSIS: Abdominal wall abscess. PROCEDURE: Incision and drainage (I&D) of abdominal abscess, excisional debridement of nonviable and viable skin, subcutaneous tissue and muscle, then removal of foreign body. ANESTHESIA: LMA. INDICATIONS: Patient is a pleasant 60-year-old gentleman, who initially had a sigmoid colectomy for diverticular abscess, subsequently had a dehiscence with evisceration. Came in approximately 36 hours ago with pain across his lower abdomen. CT scan demonstrated presence of an abscess beneath the incision. I recommended to the patient he undergo the above-named procedure. Procedure, purpose, risks, expected benefits, potential complications, alternatives forms of therapy were discussed with him, and he was agreeable to surgery. FINDINGS: The patient was found to have an abscess that went down to the level of the fascia. The anterior layer of the fascia was fibrinous and some portions necrotic. This was excisionally debrided using the Bovie cautery, and there were multiple pieces of suture within the wound and these were removed as well. TECHNIQUE: Patient was identified, then taken into the operating room, where after induction of appropriate anesthesia, his abdomen was prepped with Betadine solution and draped in a sterile fashion. The wound opening where it was draining was explored using a curette. The extent of the wound marked with a marking pen and using the Bovie cautery, the abscess was opened and drained. I then noted that there was a significant amount of undermining. These margins were marked with a marking pen, excised with Bovie cautery; the curette was used to remove the necrotic fascia. The wound was irrigated; cultures sent prior to irrigation and after achievement of excellent hemostasis, the wound was packed with antibiotic-soaked gauze. A dressing was applied. The finished wound size was 9. 0 x 5. 3 x 5. 2 cm in size. Patient tolerated the procedure well. Dressing was applied, and he was taken to recovery room in stable condition."
bbb = "PREOPERATIVE DIAGNOSIS: Recurrent re-infected sebaceous cyst of abdomen. POSTOPERATIVE DIAGNOSES:1. Abscess secondary to retained foreign body. 2. Incisional hernia. PROCEDURES1. Excision of abscess, removal of foreign body. 2. Repair of incisional hernia. ANESTHESIA: LMA. INDICATIONS: Patient is a pleasant 37-year-old gentleman who has had multiple procedures including a laparotomy related to trauma. The patient has had a recurrently infected cyst of his mass at the superior aspect of his incision, which he says gets larger and then it drains internally, causing him to be quite ill. He presented to my office and I recommended that he undergo exploration of this area and removal. The procedure, purpose, risks, expected benefits, potential complications, and alternative forms of therapy were discussed with him and he was agreeable to surgery. FINDINGS: The patient was found upon excision of the cyst that it contained a large Prolene suture, which is multiply knotted as it always is; beneath this was a very small incisional hernia, the hernia cavity, which contained omentum; the hernia was easily repaired. DESCRIPTION OF PROCEDURE: The patient was identified, then taken into the operating room, where after induction of an LMA anesthetic, his abdomen was prepped with Betadine solution and draped in sterile fashion. The puncta of the wound lesion was infiltrated with methylene blue and peroxide. The lesion was excised and the existing scar was excised using an ellipse and using a tenotomy scissors, the cyst was excised down to its base. In doing so, we identified a large Prolene suture within the wound and followed this cyst down to its base at which time we found that it contained omentum and was in fact overlying a small incisional hernia. The cyst was removed in its entirety, divided from the omentum using a Metzenbaum and tying with 2-0 silk ties. The hernia repair was undertaken with interrupted 0 Vicryl suture with simple sutures. The wound was then irrigated and closed with 3-0 Vicryl subcutaneous and 4-0 Vicryl subcuticular and Steri-Strips. Patient tolerated the procedure well. Dressings were applied and he was taken to recovery room in stable condition."
annotation = model.predict(bbb)
#get_bows([aaa])

In [10]:
list(annotation)

[('Drug', 1405, 1413, 'peroxide'),
 ('Drug', 2016, 2022, 'Vicryl'),
 ('Route', 2023, 2035, 'subcutaneous')]

In [13]:
dat = pd.read_csv("mtsample_gastroenterology_neurology.csv")
notes = list(dat.medical_note)

In [14]:
# an annotation looks like:
# [('Drug', 1405, 1413, 'peroxide'),
# ('Drug', 2016, 2022, 'Vicryl'),
# ('Route', 2023, 2035, 'subcutaneous')]
# So the element of index 3 is the keyword
def get_bows(notes):
    # notes: medical notes as a list of strings
    bows = []
    for note in tqdm(notes):
        annotation = model.predict(note)
        bow = [ann[3].replace(" ", "-") for ann in list(annotation)]
        bow = " ".join(bow)
        bows.append(bow)
    return(bows)

In [15]:
bows = get_bows(notes)

100%|██████████| 451/451 [01:07<00:00,  6.73it/s]


In [7]:
# save bows which take 13 min to run
with open("bows_medacy_gas_neu.txt", "w") as f:
    json.dump(bows, f)
    
with open("bows_medacy_gas_neu.txt", "r") as f:
    if "bows" not in globals():
        bows = json.load(f)

In [18]:
bows_gas_neu_df = pd.DataFrame({
    "id" : range(1, 452),
    "medacy_me" : bows
})
bows_gas_neu_df.to_csv("medacy_medical_entities.csv", index=False)

In [17]:
bows

['foreign-body',
 'lymphatic-leak chlorhexidine Subcuticular-5-0 Monocryl skin-closure 100-mL',
 'oral antibiotics Cipro Flagyl atrial-fibrillation metoprolol Premarin-hormone replacement. creatinine 0.',
 'peroxide Vicryl subcutaneous',
 'right-upper-quadrant-pain calcium antibiotics',
 'Vicryl Vicryl',
 'Ativan hyponatremia',
 'Vicryl saline Vicryl Vicryl benzoin',
 'Acute-appendicitis Acute-appendicitis anesthesia local-peritonitis Vicryl Z-stitch Vicryl normal-saline fine-Monocryl',
 'Acute-appendicitis anesthesia antibiotics saline right-upper-quadrant Vicryl Vicryl antibiotics',
 'Acute-appendicitis Acute-appendicitis acute-appendicitis saline-antibiotic',
 'Acute-appendicitis Acute-suppurative-appendicitis Marcaine acute-appendicitis acute-appendicitis Marcaine 0.25 antibiotics',
 'Barium-enema constipation barium-contrast',
 'chemotherapy April decreased-leakage-from-his-trach Passy chemotherapy increase-laryngeal-movement',
 'TIA/stroke bolus',
 'bolus bolus',
 'bolus decrease

## Machine learning with bag of words

In [15]:
# add a new column bow to the data
dat["bow"] = bows
bow_df = dat[["sample_type", "bow"]]
bow_df

,sample_type,bow
0,Allergy / Immunology,"[Claritin, Zyrtec, Zyrtec, loratadine, Nasonex..."
1,Allergy / Immunology,"[perioral-swelling, allergy-reaction, Keflex, ..."
2,Allergy / Immunology,"[prednisone, antibiotic, Advair, Nasonex, Xope..."
3,Allergy / Immunology,"[chronic-sinusitis, antibiotics, Veramyst, nas..."
4,Allergy / Immunology,[peptic-ulcer-disease]
...,...,...
4994,Urology,[local-anesthesia]
4995,Urology,[Vicryl]
4996,Urology,"[lidocaine, anesthesia, Tylenol, Codeine, pain]"
4997,Urology,"[Betadine, 2, Xylocaine, 2, Xylocaine, Bacitra..."
